In [33]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB3 --baud=115200 
Ready.


In [34]:
from machine import I2C, Pin
i2c = I2C(scl=Pin(15, Pin.OUT, Pin.PULL_UP), sda=Pin(4, Pin.OUT, Pin.PULL_UP))

ads = i2c.scan()
print([hex(s)  for s in ads])
if 0x68 in ads:
    print("We have an MPU6050")


['0x68']
We have an MPU6050


In [35]:
%sendtofile --source MPU6050_funcs.py
#%sendtofile --source MPU6050_dmpMemoryv6.hex

Sent 163 lines (6486 bytes) to MPU6050_funcs.py.


In [36]:
from MPU6050_funcs import setupDMP, calibgyros, calibacc

setupDMP(i2c, "MPU6050_dmpMemoryv6.hex")
calibgyros((80, -14, 30))
calibacc((582, 951, 1692))


In [40]:
writeBit(0x6A, 7, 1)
print(hex(readByte(0x6A)))
writeBit(0x6A, 7, 0)


0xc0


In [46]:
import ubinascii
from MPU6050_funcs import setupDMP, calibgyros, calibacc
from MPU6050_funcs import getFIFOCount, setDMPEnabled, getIntStatus, readfifoBuffer, ConvertQuatToEuler

import time
setDMPEnabled(True)
for i in range(50):
    mpuIntStatus = getIntStatus()
    fifoCount = getFIFOCount()
    if fifoCount >= 28:
        fifoBuffer = readfifoBuffer(28)
        #print(ubinascii.hexlify(fifoBuffer))
        continue
    print(mpuIntStatus, fifoCount)
    time.sleep_ms(4)
print(getFIFOCount())
setDMPEnabled(False)
mpuIntStatus = getIntStatus()
fifoCount = getFIFOCount()
print(mpuIntStatus, fifoCount)

3 0
3 0
3 0
1 0
1 0
1 0
3 0
1 0
3 0
3 0
3 0
1 0
3 0
1 0
3 0
1 0
3 0
1 0
3 0
1 0
0
3 0


In [52]:
from MPU6050_funcs import writeBit, readByte, writeByte, ConvertQuatToEuler
import time, ustruct 

setDMPEnabled(True)
i = 0
while i < 1500:
    fifoBuffer = None
    while getFIFOCount() >= 28:
        fifoBuffer = readfifoBuffer(28)
    if fifoBuffer is not None:
        i += 1
        if (i%50) == 1:
            q0, l, q1, l, q2, l, q3, l = ustruct.unpack(">hhhhhhhh", fifoBuffer)
            print(ConvertQuatToEuler(q0, q1, q2, q3))
    else:
        time.sleep_ms(4)
print(getFIFOCount())
setDMPEnabled(False)


(81.79306, -0.1611739, 225.7938)
(76.2696, 3.291496, 139.0428)
(78.63178, 7.121366, 54.07476)
(84.30116, 1.607184, 287.4115)
(73.21054, -2.392029, 345.5811)
(55.37981, 34.46194, 312.9005)
(73.87878, 7.215528, 355.9018)
(67.95148, 22.0149, 299.2634)
(77.65548, 8.898805, 321.3654)
(69.52027, -11.18773, 244.2555)
(41.34737, -47.43162, 238.6008)
(23.95605, 51.93978, 241.3801)
(43.97464, -22.02891, 245.8946)
(36.10372, 33.83172, 214.0178)
(53.05242, -5.902741, 219.9172)
(15.20683, 24.26231, 287.3217)
(25.31234, -6.249844, 235.4435)
(42.82795, -22.46389, 260.9643)
(80.50888, 8.770383, 293.098)
(72.19625, -6.192444, 48.42065)
(67.64935, -19.46391, 72.34277)
(59.30353, 11.81353, 55.69047)
(68.00523, 21.94616, 18.50809)
(87.61903, -2.255959, 75.18922)
(71.51754, 1.970032, 84.05183)
(70.54081, 15.43255, 30.72014)
(82.2402, 7.503702, 306.608)
(79.03867, 1.89553, 131.5143)
(73.83565, 9.732336, 54.33524)
(78.31604, 9.826788, 329.4065)
0
